In [1]:
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from utils import read_config, MySQLAgent
import time, random

In [2]:
config = read_config('.env/local_conn.json')
sql_agent = MySQLAgent(config['local_mysql'])

In [3]:
query = """
show tables
"""

sql_agent.read_table(query=query)

,tables_in_generaldb
0,balancesheet


### Helper functions

In [89]:
def get_balancesheet_data_and_rename(df, row_name, new_name):

    result = df[df['acct_name'] == row_name][['year_season', 'this_year_amt']].reset_index(drop=True).rename(columns={'this_year_amt':new_name})
    result[new_name] = result[new_name].str.replace(',', '').astype(int)

    return result

In [39]:
query = """
select *
from balancesheet
where stock = '2330' and season = 4
"""

df_balance = sql_agent.read_table(query=query)

In [40]:
df_balance['year_season'] = df_balance['period_year'].astype(str) + '-' + df_balance['season'].astype(str)

In [41]:
df_balance.acct_name.unique()

array(['流動資產', '現金及約當現金', '透過損益按公允價值衡量之金融資產－流動',
       '透過其他綜合損益按公允價值衡量之金融資產－流動', '按攤銷後成本衡量之金融資產－流動', '避險之金融資產－流動',
       '應收帳款淨額', '應收帳款－關係人淨額', '其他應收款－關係人淨額', '存貨', '其他流動資產', '流動資產合計',
       '非流動資產', '透過其他綜合損益按公允價值衡量之金融資產－非流動', '按攤銷後成本衡量之金融資產－非流動',
       '採用權益法之投資', '不動產、廠房及設備', '使用權資產', '無形資產', '遞延所得稅資產', '其他非流動資產',
       '非流動資產合計', '資產總額', '流動負債', '短期借款', '透過損益按公允價值衡量之金融負債－流動',
       '避險之金融負債－流動', '應付帳款', '應付帳款－關係人', '其他應付款', '本期所得稅負債', '其他流動負債',
       '流動負債合計', '非流動負債', '應付公司債', '遞延所得稅負債', '租賃負債－非流動', '其他非流動負債',
       '非流動負債合計', '負債總額', '歸屬於母公司業主之權益', '股本', '普通股股本', '股本合計', '資本公積',
       '資本公積－發行溢價', '資本公積-認列對子公司所有權權益變動數', '資本公積－受贈資產',
       '資本公積－採用權益法認列關聯企業及合資股權淨值之變動數', '資本公積－合併溢額', '資本公積合計', '保留盈餘',
       '法定盈餘公積', '特別盈餘公積', '未分配盈餘（或待彌補虧損）', '保留盈餘合計', '其他權益', '其他權益合計',
       '歸屬於母公司業主之權益合計', '非控制權益', '權益總額', '負債及權益總計',
       '預收股款（權益項下）之約當發行股數（單位：股）', '母公司暨子公司所持有之母公司庫藏股股數（單位：股）', '長期借款',
       '資本公積－實際取得或處分子公司股權價格與帳面價值差額', '資本公積－限制員工權利股票', '庫藏股票',
       '透過損益按公允價

### DBR

In [91]:
# 資產總額
df_total_assests = get_balancesheet_data_and_rename(df_balance, '資產總額', 'total_assests')
df_total_assests['total_assests_varation'] = df_total_assests['total_assests'].pct_change()*100
df_total_assests

,year_season,total_assests,total_assests_varation
0,108-4,2264805032,NaN
1,109-4,2760711405,21.896206
2,110-4,3725503455,34.947226
3,111-4,4964778878,33.264643
4,112-4,5532371215,11.432379


In [92]:
# 負債總額
df_total_liability = get_balancesheet_data_and_rename(df_balance, '負債總額', 'total_liabilities')
df_total_liability['total_liabilities_varation'] = df_total_liability['total_liabilities'].pct_change()*100
df_total_liability

,year_season,total_liabilities,total_liabilities_varation
0,108-4,642709606,NaN
1,109-4,910089406,41.601961
2,110-4,1554770250,70.837089
3,111-4,2004290011,28.912295
4,112-4,2049108368,2.236121


In [116]:
# DBR: Debut Burden Ratio
df_total_assests_liability = pd.merge(df_total_assests, df_total_liability, how='inner', on='year_season')
df_total_assests_liability['DBR'] = (round(df_total_assests_liability['total_liabilities'].astype(int) / df_total_assests_liability['total_assests'].astype(int) * 100, 2)).astype(str) + '%'
df_total_assests_liability

,year_season,total_assests,total_assests_varation,total_liabilities,total_liabilities_varation,DBR
0,108-4,2264805032,NaN,642709606,NaN,28.38%
1,109-4,2760711405,21.896206,910089406,41.601961,32.97%
2,110-4,3725503455,34.947226,1554770250,70.837089,41.73%
3,111-4,4964778878,33.264643,2004290011,28.912295,40.37%
4,112-4,5532371215,11.432379,2049108368,2.236121,37.04%


- 108年~110年，債務比例增加，可能是要擴廠或增加設備等等

### 長期資金佔固定資產比率(Long term funds to fixed assets ratio)

(股東權益淨額 + 長期負債) / 固定資產淨額

在評估企業財務穩健性時，長期資金佔固定資產比率是一個重要的指標。長期資金，如股本和長期負債，被視為相對穩定的資金來源，而固定資產是指長期持有、無法迅速變現的資產，如廠房和設備。長期資金佔固定資產比率衡量長期資金對固定資產的覆蓋程度，反映企業償還固定資產投資的能力。

一般而言，較高的長期資金佔固定資產比率表示企業的財務結構較為穩健，因為它意味著企業有更多資金可供償還固定資產投資。然而，過高的比率也並非好事。當長期資金過多時，企業可能會無法有效利用資金，導致資金利用率低下，進而影響盈利能力。因此，在評估企業的財務穩健性時，投資者需要綜合考量長期資金佔固定資產比率和其他財務指標，例如負債對股東權益比率和現金流量狀況，才能做出全面判斷。

In [94]:
# 固定資產: 不動產、廠房及設備
df_fixed_assets = get_balancesheet_data_and_rename(df_balance, '不動產、廠房及設備', 'fixed_assests')
df_fixed_assets

,year_season,fixed_assests
0,108-4,1352377405
1,109-4,1555589120
2,110-4,1975118704
3,111-4,2693836970
4,112-4,3064474984


In [95]:
# 長期借款
df_long_term_liabilities = get_balancesheet_data_and_rename(df_balance, '長期借款', 'long-term_liabilities')
df_long_term_liabilities['']

,year_season,long-term_liabilities
0,109-4,1967611
1,110-4,3309131
2,111-4,4760047
3,112-4,4382965


In [101]:
df_balance[df_balance['acct_name'] == '長期借款']

,acct_name,this_year_amt,this_year_percent,last_year_amt,last_year_percent,report_name,report_time_raw,period_year,season,creation_date,stock,year_season
99,長期借款,"1,967,611",0.07,0,0.00,BalanceSheet,民國109年第4季,109,4,2024-08-05,2330,109-4
164,長期借款,"3,309,131",0.09,"1,967,611",0.07,BalanceSheet,民國110年第4季,110,4,2024-08-05,2330,110-4
230,長期借款,"4,760,047",0.10,"3,309,131",0.09,BalanceSheet,民國111年第4季,111,4,2024-08-05,2330,111-4
298,長期借款,"4,382,965",0.08,"4,760,047",0.10,BalanceSheet,民國112年第4季,112,4,2024-08-05,2330,112-4


In [96]:
# 股東權益＝資產－負債
df_shareholder_equity = df_total_assests_liability[['year_season', 'total_assests', 'total_liabilities']].copy()
df_shareholder_equity['shareholder_equity'] = df_shareholder_equity['total_assests'] - df_shareholder_equity['total_liabilities']
df_shareholder_equity

,year_season,total_assests,total_liabilities,shareholder_equity
0,108-4,2264805032,642709606,1622095426
1,109-4,2760711405,910089406,1850621999
2,110-4,3725503455,1554770250,2170733205
3,111-4,4964778878,2004290011,2960488867
4,112-4,5532371215,2049108368,3483262847


In [113]:
parent_table = df_balance[['year_season']].drop_duplicates()
df_temp1 = pd.merge(df_fixed_assets, df_long_term_liabilities, how='left', on='year_season')
df_temp2 = pd.merge(df_temp1, df_shareholder_equity, how='left', on='year_season')
# in here, we should check the last amt in the original table. 0 here is the result after checking
# create a check function laterly
df_longterm_funds_to_fixed_assets_retio = pd.merge(parent_table, df_temp2, how='left', on='year_season').fillna(0)
df_longterm_funds_to_fixed_assets_retio['longterm_funds_to_fixed_assets_retio'] = round((df_longterm_funds_to_fixed_assets_retio['long-term_liabilities'].astype(int) + df_longterm_funds_to_fixed_assets_retio['shareholder_equity'])/ df_longterm_funds_to_fixed_assets_retio['fixed_assests'].astype(int), 2)
df_longterm_funds_to_fixed_assets_retio

,year_season,fixed_assests,long-term_liabilities,total_assests,total_liabilities,shareholder_equity,longterm_funds_to_fixed_assets_retio
0,108-4,1352377405,0.0,2264805032,642709606,1622095426,1.20
1,109-4,1555589120,1967611.0,2760711405,910089406,1850621999,1.19
2,110-4,1975118704,3309131.0,3725503455,1554770250,2170733205,1.10
3,111-4,2693836970,4760047.0,4964778878,2004290011,2960488867,1.10
4,112-4,3064474984,4382965.0,5532371215,2049108368,3483262847,1.14


- The longterm_funds_to_fixed_assets_ratio for TSMC has fluctuated slightly between 1.10 and 1.20 over the past five years, indicating a balanced and consistent financial strategy.
- The slight decrease in the ratio in 2021 and 2022 reflects strategic investments in fixed assets, possibly for expansion or upgrading capabilities.
- The increase in the ratio in 2023 suggests a return to a slightly more conservative approach, increasing long-term funds relative to fixed assets, which enhances financial stability.
- The consistent ratios indicate that TSMC effectively manages its long-term funds to ensure sufficient coverage for its fixed assets, supporting sustained growth and operational resilience.